In [18]:
import json
import pysal as ps
import urllib

In [2]:
u = urllib.urlopen("http://toae.org/pub/columbus.json")

In [3]:
u

<addinfourl at 140486997678776 whose fp = <socket._fileobject object at 0x7fc5cc8284d0>>

In [4]:
u.url

'http://toae.org/pub/columbus.json'

In [5]:
g = json.load(u)

In [6]:
g.keys()

[u'type', u'features']

In [7]:
t = g['type']

In [8]:
t

u'FeatureCollection'

In [9]:
f = g['features']

In [10]:
len(f)

49

In [16]:
def contiguity_from_geojson(uri, wtype='rook'):
    WTYPE = wtype.upper()
    if WTYPE not in ['QUEEN', 'ROOK']:
        print "wtype must be 'QUEEN' or 'ROOK'"
    WTYPE =["QUEEN", 'ROOK'].index(WTYPE)+1
    collection = json.load(urllib.urlopen(uri))
    n = len(collection)
    collection_type = collection['features'][0]['geometry']['type']
    if collection_type in ['Polygon', 'MultiPolygon']:
        polys = []
        ids = []
        i = 0
        for feature in collection['features']:
            #print feature
            polys.append(ps.cg.asShape(feature['geometry']))
            ids.append(i)
            i +=1
        pcollection = ps.cg.shapes.PolygonCollection(dict(zip(ids,polys)))
        neighbors = ps.weights.Contiguity.ContiguityWeightsPolygons(pcollection, wttype=WTYPE).w
        return ps.W(neighbors)
    else:
        print "GeoJSON feature type must be 'Polygon' or 'MultiPolygon'"
       
def queen_from_geojson(uri):
    return contiguity_from_geojson(uri, wtype='QUEEN')

def rook_from_geojson(uri):
    return contiguity_from_geojson(uri, wtype="ROOK")
    

In [17]:
wr = rook_from_geojson("http://toae.org/pub/columbus.json")
wq = queen_from_geojson("http://toae.org/pub/columbus.json")

AttributeError: 'module' object has no attribute 'weights'

In [ ]:
wq.n

In [ ]:
wqps = ps.queen_from_shapefile(ps.examples.get_path("columbus.shp"))

In [ ]:
wqps.n

In [ ]:
wq.n

In [ ]:
wq[0]

In [ ]:
wqps[0]

In [ ]:
wrps = ps.rook_from_shapefile(ps.examples.get_path('columbus.shp'))

In [ ]:
wrps[0]

In [ ]:
wr[0]

In [ ]:
wr

In [ ]:
wr.neighbors

In [ ]:
wqps.neighbors.keys() == wq.neighbors.keys()

In [ ]:
for key in wqps.neighbors.keys():
    if wqps[key] != wq[key]:
        print key, wqps[key], wq[key]

In [ ]:
wrps.neighbors.keys() == wr.neighbors.keys()

In [ ]:
for key in wrps.neighbors.keys():
    if wrps[key] != wr[key]:
        print key, wrps[key], wr[key]

# string versus file stream

In [ ]:
strng = json.dumps(json.load(urllib.urlopen("http://toae.org/pub/columbus.json")),separators=(',',':') )

In [ ]:
fc = json.loads(strng)

In [ ]:
fc

In [ ]:
def _geojson_load(gj_input):
    if type(gj_input) == str:
        return json.loads(gj_input)
    else:
        return json.load(gj_input)
    
fs = urllib.urlopen("http://toae.org/pub/columbus.json")

g1 = _geojson_load(fs)


ss = json.dumps(json.load(urllib.urlopen("http://toae.org/pub/columbus.json")),separators=(',',':') )

g2 = _geojson_load(ss)

g1 == g2

In [ ]:
def contiguity_from_geojson(gj_input, wtype='rook'):
    WTYPE = wtype.upper()
    if WTYPE not in ['QUEEN', 'ROOK']:
        print "wtype must be 'QUEEN' or 'ROOK'"
    WTYPE =["QUEEN", 'ROOK'].index(WTYPE)+1
    collection = _geojson_load(gj_input)
    #collection = json.load(urllib.urlopen(uri))
    n = len(collection)
    collection_type = collection['features'][0]['geometry']['type']
    if collection_type in ['Polygon', 'MultiPolygon']:
        polys = []
        ids = []
        i = 0
        for feature in collection['features']:
            #print feature
            polys.append(ps.cg.asShape(feature['geometry']))
            ids.append(i)
            i +=1
        pcollection = ps.cg.shapes.PolygonCollection(dict(zip(ids,polys)))
        neighbors = ps.weights.Contiguity.ContiguityWeightsPolygons(pcollection, wttype=WTYPE).w
        return ps.W(neighbors)
    else:
        print "GeoJSON feature type must be 'Polygon' or 'MultiPolygon'"
       
def queen_from_geojson(uri):
    return contiguity_from_geojson(uri, wtype='QUEEN')

def rook_from_geojson(uri):
    return contiguity_from_geojson(uri, wtype="ROOK")
    

In [ ]:
fs = urllib.urlopen("http://toae.org/pub/columbus.json")
